# Subgraph Queries with DataStreams

The goal of this notebook is to show and explain how to create a decentralized data pipeline powered by Cow Subgraphs with DataStreams. Right now DataStreams acts only as a GraphQL query manager. Should DataStreams implement any preprocessing or should that be done outside of the library? Currently I am leaning towards finishing it outside of the library but still in the jupyter notebook. 

2.4.23 What needs to happen to keep the preprocessing as simple as possible to fit in a single notebook? How would I describe my current preprocessing approach and what has been done thus far?

Subgraph Link here - https://thegraph.com/hosted-service/subgraph/cowprotocol/cow
Dune Query link - https://dune.com/queries/1941061

### TODO 
- Why does settlements.trades and trades queries return different values???

## Setup Jupyter Environment

First install DataStreams with `!pip install git+https://github.com/Evan-Kim2028/DataStreams.git` in jupyter or `pip install git+https://github.com/Evan-Kim2028/DataStreams.git` in a virtual environment. The primary DataStreams dependencies are Python 3.10, Subgrounds, and Pandas. 

In [5]:
!pip install git+https://github.com/Evan-Kim2028/DataStreams.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/Evan-Kim2028/DataStreams.git to /tmp/pip-req-build-oh05obci
  Running command git clone --filter=blob:none --quiet https://github.com/Evan-Kim2028/DataStreams.git /tmp/pip-req-build-oh05obci
  Resolved https://github.com/Evan-Kim2028/DataStreams.git to commit 587b2769b0431593dfffa10477cde33dbf65e705
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for DataStreams: filename=DataStreams-0.0.post1.dev61+g587b276-py3-none-any.whl size=8572 sha256=45441e7018c9ea8d4e8a026d9f98777e831baa2ce932ee176bca1a568096a70e
  Stored in directory: /tmp/pip-ephem-wheel-cache-zzzcdmnx/wheels/06/e7/63/91de055318e05be20cb8ce687f960fcb1ae3a9e77f754cda6c
Successfully built DataStreams
  Attempting uninstall: DataStreams
    Found existing installation: DataStream

In [1]:
from datastreams.datastream import Streamer

import os
import pandas as pd
import polars as pl

In [2]:
# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [3]:
# instantiate Streamer classW
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

In [4]:
# DEFINE TIMESTAMP HERE
timestamp = 1650000000

### Cowswap Trades Schema

In [5]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
trades_fp = cow_ds.queryDict.get('trades')

# add parameters to the query_path
trades_qp = trades_fp(
    first=1000,
    orderBy='timestamp',
    orderDirection='desc',
    where = {'timestamp_lt': timestamp, 'buyAmountUsd_gt': 250, 'sellAmountUsd_gt': 250}
    )

# run query
trades_df = cow_ds.runQuery(trades_qp)

FIELD - trades


In [6]:
print(f'query returned {len(trades_df)} rows')
trades_df.head(10)

query returned 1000 rows


,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,endpoint
0,0x60181be6aa27e0735ce2ecf8045af841d304f707e8451908ac95f76b4890c58a58737a9b0a8a5c8a2559c457fcc9e923f9d99fb7625905a5|0xf402c5980e60a0339205a7012007770d8277d53abf030d56ceb74c74cb506bb4|2,1649999570,33515804349,9370499,0xf402c5980e60a0339205a7012007770d8277d53abf030d56ceb74c74cb506bb4,0xf402c5980e60a0339205a7012007770d8277d53abf030d56ceb74c74cb506bb4,6997041016,7000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xdac17f958d2ee523a2206206994597c13d831ec7,0x60181be6aa27e0735ce2ecf8045af841d304f707e8451908ac95f76b4890c58a58737a9b0a8a5c8a2559c457fcc9e923f9d99fb7625905a5,2.303213,2.304187,6997.041016,7000.000000,cow
1,0xf112b69d162117311070d2deca4828956b747b8aa74732f0abf342961b4a977558737a9b0a8a5c8a2559c457fcc9e923f9d99fb762590588|0xea3fce585e4d03aa05ac8a1f676ade26c062ddc1995cf6caf20cdc5c8a6c8143|21,1649999528,33197137892,9370496,0xea3fce585e4d03aa05ac8a1f676ade26c062ddc1995cf6caf20cdc5c8a6c8143,0xea3fce585e4d03aa05ac8a1f676ade26c062ddc1995cf6caf20cdc5c8a6c8143,6990153497537838352206,7000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x6b175474e89094c44da98b954eedeac495271d0f,0xf112b69d162117311070d2deca4828956b747b8aa74732f0abf342961b4a977558737a9b0a8a5c8a2559c457fcc9e923f9d99fb762590588,2.300946,2.304187,6990.153498,7000.000000,cow
2,0x56a04b2c409e31083e4817f0570cf7f359fbce2c820876af59467c3acea66b60e99ca63d7bd8297b6f03208e0e330589536e48e56259056d|0x91af4d1d35783b74ae1c18e83b6c9f2d2f6e102d45d8d10deec927ac6b3105e6|2,1649999505,39308808997,9370280,0x91af4d1d35783b74ae1c18e83b6c9f2d2f6e102d45d8d10deec927ac6b3105e6,0x91af4d1d35783b74ae1c18e83b6c9f2d2f6e102d45d8d10deec927ac6b3105e6,160798023556263702,500000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x56a04b2c409e31083e4817f0570cf7f359fbce2c820876af59467c3acea66b60e99ca63d7bd8297b6f03208e0e330589536e48e56259056d,0.160798,0.164585,488.495937,500.000000,cow
3,0x6e3ed88711d8ce48eec71df827e93289253758ceaa2301ff58e5b45932f1989d962228a90eac69238c7d1f216d80037e61ea925562590208|0xe359cac50f6c223600c701b134144590f5a043eb3d22616c13aae9e5470c5788|9,1649998655,38914800896,11195275808097705984,0xe359cac50f6c223600c701b134144590f5a043eb3d22616c13aae9e5470c5788,0xe359cac50f6c223600c701b134144590f5a043eb3d22616c13aae9e5470c5788,7000000000000000000000,7003249164081900947869,0x865377367054516e17014ccded1e7d814edc9ce4,0x6b175474e89094c44da98b954eedeac495271d0f,0x6e3ed88711d8ce48eec71df827e93289253758ceaa2301ff58e5b45932f1989d962228a90eac69238c7d1f216d80037e61ea925562590208,2.304187,2.303013,7000.000000,6996.331518,cow
4,0x59b35320a0fc56c3c4d18c287e65a88fd2518b0bd03550dc4f955165f1b4559f723f61815066769fc3ec7cf50cc65904496a7b816258ff63|0x4375bc61e713b3401826368a42d2d8b84e297f3107cdc14bbcc8061eb0b9044e|2,1649998556,36942918877,21347096160580600,0x4375bc61e713b3401826368a42d2d8b84e297f3107cdc14bbcc8061eb0b9044e,0x4375bc61e713b3401826368a42d2d8b84e297f3107cdc14bbcc8061eb0b9044e,77651337654462290456692,40000000000000000000,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0x6b175474e89094c44da98b954eedeac495271d0f,0x59b35320a0fc56c3c4d18c287e65a88fd2518b0bd03550dc4f955165f1b4559f723f61815066769fc3ec7cf50cc65904496a7b816258ff63,25.560462,25.755831,77651.337654,78244.859268,cow
5,0xde39ceeeef5353de54949724e72f0fb33afbd67a875a11a8d6a9edc68fb5abdd933258bddd49beeca77f6d1889633c5429af45ea6258faf3|0x4ea0496471e086f45d5d192bb87a9ee8a96d4cc294918d0f2566e08ef128db86|12,1649998237,34547566497,8436449,0x4ea0496471e086f45d5d192bb87a9ee8a96d4cc294918d0f2566e08ef128db86,0x4ea0496471e086f45d5d192bb87a9ee8a96d4cc294918d0f2566e08ef128db86,69425654684866987916,210872822710,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0xde39ceeeef5353de54949724e72f0fb33afbd67a875a11a8d6a9edc68

### Cowswap Trades-Settlement Merge

In [7]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
settlements_fp = cow_ds.queryDict.get('settlements')

# add parameters to the settlements_qp.
settlements_qp = settlements_fp(
    first=1000,
    orderBy='firstTradeTimestamp',
    orderDirection='desc',
    where = {'firstTradeTimestamp_lt': timestamp} 
    )

# run query
settlements_df = cow_ds.runQuery(settlements_qp)

FIELD - settlements


In [8]:
settlements_df

,settlements_id,settlements_txHash,settlements_firstTradeTimestamp,settlements_solver_id,endpoint
0,0x7032f3449976b7a4c18983318597b69bbce0e27d6d16c47a471bfcb09c9f84fc,0x7032f3449976b7a4c18983318597b69bbce0e27d6d16c47a471bfcb09c9f84fc,1649999950,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,cow
1,0x95c3a6f58df920b15ec2b0ef6d177105874826da4f820397aac8c9bb8b4e077b,0x95c3a6f58df920b15ec2b0ef6d177105874826da4f820397aac8c9bb8b4e077b,1649999841,0x77ec2a722c2393d3fd64617bbaf1499c713e616b,cow
2,0xf402c5980e60a0339205a7012007770d8277d53abf030d56ceb74c74cb506bb4,0xf402c5980e60a0339205a7012007770d8277d53abf030d56ceb74c74cb506bb4,1649999570,0xe33062a24149f7801a48b2675ed5111d3278f0f5,cow
3,0xea3fce585e4d03aa05ac8a1f676ade26c062ddc1995cf6caf20cdc5c8a6c8143,0xea3fce585e4d03aa05ac8a1f676ade26c062ddc1995cf6caf20cdc5c8a6c8143,1649999528,0xda324c2f06d3544e7965767ce9ca536dcb67a660,cow
4,0x91af4d1d35783b74ae1c18e83b6c9f2d2f6e102d45d8d10deec927ac6b3105e6,0x91af4d1d35783b74ae1c18e83b6c9f2d2f6e102d45d8d10deec927ac6b3105e6,1649999505,0x77ec2a722c2393d3fd64617bbaf1499c713e616b,cow
...,...,...,...,...,...
995,0xf2eced7ccfc706035b6b05d8ceca0e7b037248d55c4ba13d2e8273bf9ee0f0ab,0xf2eced7ccfc706035b6b05d8ceca0e7b037248d55c4ba13d2e8273bf9ee0f0ab,1649876410,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,cow
996,0x0ce10411d987b48ac3f449c72b72a5aa2205aba21789dd481f075de3642e9cfa,0x0ce10411d987b48ac3f449c72b72a5aa2205aba21789dd481f075de3642e9cfa,1649876410,0xe33062a24149f7801a48b2675ed5111d3278f0f5,cow
997,0xb6e0fab841eb5248d2d4534b6a2ccfd86154be703f2fff25a5a32bd2a11e28ed,0xb6e0fab841eb5248d2d4534b6a2ccfd86154be703f2fff25a5a32bd2a11e28ed,1649876283,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,cow
998,0x05a21b93b9d38aa92f854aeb78f5709ddee01d65d67e46ea20675e453662b0df,0x05a21b93b9d38aa92f854aeb78f5709ddee01d65d67e46ea20675e453662b0df,1649876089,0xe92f359e6f05564849afa933ce8f62b8007a1d5d,cow


In [9]:
# convert dfs into a dictionaries
settlement_dict = settlements_df.to_dict('records')
trades_dict = trades_df.to_dict('records')

In [10]:
settlement_pl = pl.from_dicts(settlement_dict)
trades_pl = pl.from_dicts(trades_dict)

In [11]:
trades_pl.columns

['trades_id',
 'trades_timestamp',
 'trades_gasPrice',
 'trades_feeAmount',
 'trades_txHash',
 'trades_settlement_id',
 'trades_buyAmount',
 'trades_sellAmount',
 'trades_sellToken_id',
 'trades_buyToken_id',
 'trades_order_id',
 'trades_buyAmountEth',
 'trades_sellAmountEth',
 'trades_buyAmountUsd',
 'trades_sellAmountUsd',
 'endpoint']

In [12]:
settlement_pl.columns

['settlements_id',
 'settlements_txHash',
 'settlements_firstTradeTimestamp',
 'settlements_solver_id',
 'endpoint']

In [13]:
# merge trades and settlement dataframes on the settlement transaction hash
cow_trades_pl = trades_pl.join(other=settlement_pl, left_on='trades_settlement_id', right_on='settlements_txHash', how='inner')

In [14]:
cow_trades_pl.shape

(787, 20)

### Cowswap Trades-Solver Merge

In [15]:
solvers = pd.read_csv('data/cowv2_solvers.csv') # load in pandas instead of polars. Having trouble replacing \ symbol in polars

In [16]:
# rename address to settlements_solver_id in pandas
solvers = solvers.rename(columns={"address": "settlements_solver_id"})

In [17]:
# NOTE - dune formats addresses as /x... need to convert '/' to '0'
solvers['settlements_solver_id'] = solvers['settlements_solver_id'].str.replace('\\', '0', regex=False)

In [18]:
# turn solvers into a dictionary
solvers_dict = solvers.to_dict('records')

# convert dict to polars
solvers_pl = pl.from_dicts(solvers_dict)

In [19]:
# inner join solvers_pl on total_settlement_tokens_pl
cow_complete_pl = cow_trades_pl.join(solvers_pl, on="settlements_solver_id", how="inner")

In [20]:
cow_complete_pl.shape

(787, 23)

#### Basic Agg

In [21]:
# filter by "prod" environments
filter_df = cow_complete_pl.filter(pl.col("environment") == "prod")

In [22]:
# filter by "prod" environments
filter_df = cow_complete_pl.filter(pl.col("environment") == "prod")

In [23]:
filter_df.shape

(771, 23)

In [24]:
# group filter_df by solver name polars
grouped_df = filter_df.groupby("name").agg(
    pl.count("trades_id").alias("total_trades")).sort("total_trades", reverse=True)


In [25]:
grouped_df

name,total_trades
str,u32
"""Gnosis_1inch""",266
"""Gnosis_0x""",186
"""DexCowAgg""",125
"""QuasiModo""",89
"""Gnosis_ParaSwa...",61
"""MIP""",32
"""Baseline""",9
"""Gnosis_Balance...",3


### Uniswap v3

In [26]:
# instantiate Streamer object
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

In [27]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
swaps_fp = univ3_ds.queryDict.get('swaps')

In [28]:
# add parameters to the query_path
swaps_qp = swaps_fp(
    first=2001,
    orderBy='timestamp',
    orderDirection='desc',
    where = {'timestamp_lt': timestamp} 
    )

In [29]:
# run query
swaps_df = univ3_ds.runQuery(swaps_qp)

# trunc_df = swaps_df[[
#     'swaps_hash', 
#     'swaps_blockNumber', 
#     'swaps_timestamp', 
#     'swaps_tokenIn_id',
#     'swaps_tokenOut_id',
#     'swaps_amountIn',
#     'swaps_amountOut',
#     'swaps_amountInUSD',
#     'swaps_amountOutUSD',
#     ]]

FIELD - swaps


In [30]:
print(f'query returned {len(swaps_df)} rows. swaps_df columns are {swaps_df.columns}')

query returned 2001 rows. swaps_df columns are Index(['swaps_id', 'swaps_hash', 'swaps_logIndex', 'swaps_protocol_id',
       'swaps_to', 'swaps_from', 'swaps_blockNumber', 'swaps_timestamp',
       'swaps_tokenIn_id', 'swaps_amountIn', 'swaps_amountInUSD',
       'swaps_tokenOut_id', 'swaps_amountOut', 'swaps_amountOutUSD',
       'swaps_pool_id', 'endpoint'],
      dtype='object')


In [31]:
# convert swaps_df to pl
swaps_dict = swaps_df.to_dict('records')
swaps_pl = pl.from_dicts(swaps_dict)

In [32]:
swaps_pl.head(10)

swaps_id,swaps_hash,swaps_logIndex,swaps_protocol_id,swaps_to,swaps_from,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,swaps_amountIn,swaps_amountInUSD,swaps_tokenOut_id,swaps_amountOut,swaps_amountOutUSD,swaps_pool_id,endpoint
str,str,i64,str,str,str,i64,i64,str,f64,f64,str,f64,f64,str,str
"""0xd6b18367b4a3...","""0xd6b18367b4a3...",417,"""0x1f98431c8ad9...","""0x35ec38cea5e0...","""0x35ec38cea5e0...",14588051,1649999976,"""0x6b175474e890...",5.0000e21,5000.0,"""0xa0b86991c621...",4.9995e9,4999.459926,"""0x5777d92f2086...","""uniswap-v3-eth..."
"""0xce168a94da99...","""0xce168a94da99...",163,"""0x1f98431c8ad9...","""0x220bda5c8994...","""0x3628edcb16ab...",14588051,1649999976,"""0x2b591e99afe9...",1.4839e14,187051.415742,"""0xa0b86991c621...",1.8450e11,184496.640242,"""0xe05e653453f7...","""uniswap-v3-eth..."
"""0xce168a94da99...","""0xce168a94da99...",160,"""0x1f98431c8ad9...","""0x220bda5c8994...","""0x3628edcb16ab...",14588051,1649999976,"""0x2b591e99afe9...",2.0235e13,25507.011237,"""0xa0b86991c621...",2.5252e10,25252.35264,"""0x69d91b94f0aa...","""uniswap-v3-eth..."
"""0xce168a94da99...","""0xce168a94da99...",157,"""0x1f98431c8ad9...","""0x220bda5c8994...","""0x3628edcb16ab...",14588051,1649999976,"""0xc02aaa39b223...",1.7308e19,52581.618425,"""0xa0b86991c621...",5.2439e10,52439.179741,"""0x88e6a0c2ddd2...","""uniswap-v3-eth..."
"""0xce168a94da99...","""0xce168a94da99...",154,"""0x1f98431c8ad9...","""0x220bda5c8994...","""0x3628edcb16ab...",14588051,1649999976,"""0x2b591e99afe9...",4.2157e13,53139.606745,"""0xc02aaa39b223...",1.7308e19,52581.618425,"""0x9e0905249cee...","""uniswap-v3-eth..."
"""0xcd09648b81bf...","""0xcd09648b81bf...",147,"""0x1f98431c8ad9...","""0x1437033c44d4...","""0x1437033c44d4...",14588051,1649999976,"""0xa0b86991c621...",5.0000e9,5000.0,"""0x2b591e99afe9...",3.9666e12,5000.001172,"""0x69d91b94f0aa...","""uniswap-v3-eth..."
"""0xa5ab2d800ae2...","""0xa5ab2d800ae2...",94,"""0x1f98431c8ad9...","""0xdef1c0ded9be...","""0x4fde6eaa7c93...",14588051,1649999976,"""0xc5b3d3231001...",1.6129e24,500.226309,"""0xc02aaa39b223...",1.6294e17,495.005669,"""0x86347ace0fbf...","""uniswap-v3-eth..."
"""0x2d08972f6755...","""0x2d08972f6755...",102,"""0x1f98431c8ad9...","""0xdef1c0ded9be...","""0x09fe10b57ab0...",14588051,1649999976,"""0xa0b86991c621...",1.782e8,178.2,"""0xc02aaa39b223...",5.8757e16,178.501134,"""0x88e6a0c2ddd2...","""uniswap-v3-eth..."
"""0xf6e8c5dd6857...","""0xf6e8c5dd6857...",496,"""0x1f98431c8ad9...","""0xd0066d17567f...","""0xd0066d17567f...",14588050,1649999950,"""0xc02aaa39b223...",2.4991e18,7592.238373,"""0xd533a949740b...",3.3330e21,7510.558755,"""0x4c83a7f819a5...","""uniswap-v3-eth..."


In [40]:
swaps_pl.shape

(2001, 16)

In [33]:
cow_complete_pl.columns

['trades_id',
 'trades_timestamp',
 'trades_gasPrice',
 'trades_feeAmount',
 'trades_txHash',
 'trades_settlement_id',
 'trades_buyAmount',
 'trades_sellAmount',
 'trades_sellToken_id',
 'trades_buyToken_id',
 'trades_order_id',
 'trades_buyAmountEth',
 'trades_sellAmountEth',
 'trades_buyAmountUsd',
 'trades_sellAmountUsd',
 'endpoint',
 'settlements_id',
 'settlements_firstTradeTimestamp',
 'settlements_solver_id',
 'endpoint_right',
 'environment',
 'name',
 'active']

### Merge Cow and Univ3

In [34]:
# drop endpoint_right column from total_settlement_tokens_solvers
cow_complete_pl = cow_complete_pl.drop('endpoint_right')

In [35]:
# merge trades and swaps on timestamp value
cow_uni_pl = cow_complete_pl.join(other=swaps_pl, left_on='trades_timestamp', right_on='swaps_timestamp', how='inner')

In [39]:
#check pl dataframe size
cow_uni_pl.shape

(325, 37)

In [36]:
cow_uni_pl.head(5)

trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,endpoint,settlements_id,settlements_firstTradeTimestamp,settlements_solver_id,environment,name,active,swaps_id,swaps_hash,swaps_logIndex,swaps_protocol_id,swaps_to,swaps_from,swaps_blockNumber,swaps_tokenIn_id,swaps_amountIn,swaps_amountInUSD,swaps_tokenOut_id,swaps_amountOut,swaps_amountOutUSD,swaps_pool_id,endpoint_right
str,i64,i64,f64,str,str,f64,f64,str,str,str,f64,f64,f64,f64,str,str,i64,str,str,str,bool,str,str,i64,str,str,str,i64,str,f64,f64,str,f64,f64,str,str
"""0x60181be6aa27...",1649999570,33515804349,9.370499e6,"""0xf402c5980e60...","""0xf402c5980e60...",6.9970e9,7.0000e9,"""0xa0b86991c621...","""0xdac17f958d2e...","""0x60181be6aa27...",2.303213,2.304187,6997.041016,7000.0,"""cow""","""0xf402c5980e60...",1649999570,"""0xe33062a24149...","""barn""","""Gnosis_1inch""",true,"""0x3afe824fe5f0...","""0x3afe824fe5f0...",85,"""0x1f98431c8ad9...","""0x68b3465833fb...","""0xfd6a3dbfa3bf...",14588032,"""0x95392f142af1...",5.4655e22,19542.681069,"""0xc02aaa39b223...",6.1365e18,18642.313419,"""0x700f4dc28170...","""uniswap-v3-eth..."
"""0xf112b69d1621...",1649999528,33197137892,9.370496e6,"""0xea3fce585e4d...","""0xea3fce585e4d...",6.9902e21,7.0000e9,"""0xa0b86991c621...","""0x6b175474e890...","""0xf112b69d1621...",2.300946,2.304187,6990.153498,7000.0,"""cow""","""0xea3fce585e4d...",1649999528,"""0xda324c2f06d3...","""barn""","""QuasiModo""",true,"""0xa193b71f6f62...","""0xa193b71f6f62...",63,"""0x1f98431c8ad9...","""0xe85ead5cdf65...","""0xe85ead5cdf65...",14588028,"""0xc02aaa39b223...",5.0000e15,15.189737,"""0x4d224452801a...",1.2093e18,0.0,"""0xf79fc43494ce...","""uniswap-v3-eth..."
"""0xf112b69d1621...",1649999528,33197137892,9.370496e6,"""0xea3fce585e4d...","""0xea3fce585e4d...",6.9902e21,7.0000e9,"""0xa0b86991c621...","""0x6b175474e890...","""0xf112b69d1621...",2.300946,2.304187,6990.153498,7000.0,"""cow""","""0xea3fce585e4d...",1649999528,"""0xda324c2f06d3...","""barn""","""QuasiModo""",true,"""0x0e6067178804...","""0x0e6067178804...",47,"""0x1f98431c8ad9...","""0x68b3465833fb...","""0x0edad2dc65df...",14588028,"""0xa0b86991c621...",3.6458e10,36458.226541,"""0xc02aaa39b223...",1.2009e19,36481.864152,"""0x88e6a0c2ddd2...","""uniswap-v3-eth..."
"""0x56a04b2c409e...",1649999505,39308808997,9.37028e6,"""0x91af4d1d3578...","""0x91af4d1d3578...",1.6080e17,5e8,"""0xa0b86991c621...","""0xeeeeeeeeeeee...","""0x56a04b2c409e...",0.160798,0.164585,488.495937,500.0,"""cow""","""0x91af4d1d3578...",1649999505,"""0x77ec2a722c23...","""prod""","""QuasiModo""",false,"""0xf55f611fa7c6...","""0xf55f611fa7c6...",285,"""0x1f98431c8ad9...","""0x6f5d8a447985...","""0x7281251515c5...",14588026,"""0xa0b86991c621...",1.55525442e8,155.525442,"""0x6b175474e890...",1.5551e20,155.511135,"""0x5777d92f2086...","""uniswap-v3-eth..."
"""0x56a04b2c409e...",1649999505,39308808997,9.37028e6,"""0x91af4d1d3578...","""0x91af4d1d3578...",1.6080e17,5e8,"""0xa0b86991c621...","""0xeeeeeeeeeeee...","""0x56a04b2c409e...",0.160798,0.164585,488.495937,500.0,"""cow""","""0x91af4d1d3578...",1649999505,"""0x77ec2a722c23...","""prod""","""QuasiModo""",false,"""0xbed78e39f320...","""0xbed78e39f320...",116,"""0x1f98431c8ad9...","""0xdef1c0ded9be...","""0x057f8378e2b1...",14588026,"""0xdac17f958d2e...",5.0282e9,5028.163002,"""0xc02aaa39b223...",1.6567e18,5032.913208,"""0x11b815efb8f5...","""uniswap-v3-eth..."


In [37]:
cow_uni_pl.columns

['trades_id',
 'trades_timestamp',
 'trades_gasPrice',
 'trades_feeAmount',
 'trades_txHash',
 'trades_settlement_id',
 'trades_buyAmount',
 'trades_sellAmount',
 'trades_sellToken_id',
 'trades_buyToken_id',
 'trades_order_id',
 'trades_buyAmountEth',
 'trades_sellAmountEth',
 'trades_buyAmountUsd',
 'trades_sellAmountUsd',
 'endpoint',
 'settlements_id',
 'settlements_firstTradeTimestamp',
 'settlements_solver_id',
 'environment',
 'name',
 'active',
 'swaps_id',
 'swaps_hash',
 'swaps_logIndex',
 'swaps_protocol_id',
 'swaps_to',
 'swaps_from',
 'swaps_blockNumber',
 'swaps_tokenIn_id',
 'swaps_amountIn',
 'swaps_amountInUSD',
 'swaps_tokenOut_id',
 'swaps_amountOut',
 'swaps_amountOutUSD',
 'swaps_pool_id',
 'endpoint_right']

In [38]:
# save cow_complete_pl to csv
cow_uni_pl.write_csv('data/cow_uni_pl.csv')